In [1]:
%%time

import numpy as np
from scipy import sparse
import pandas as pd
from sklearn.model_selection import train_test_split
import tarfile as tar
import gzip as gz
import shutil
import os

CPU times: user 976 ms, sys: 129 ms, total: 1.1 s
Wall time: 1.69 s


In [2]:
%%time

with tar.open("Webscope_R1.tgz", "r") as tar_ref:
    tar_ref.extractall()

CPU times: user 2.02 s, sys: 1.12 s, total: 3.15 s
Wall time: 3.65 s


In [3]:
%%time

with gz.open("ydata-ymusic-user-artist-ratings-v1_0.txt.gz", "r") as f_in:
    with open('ydata-ymusic-user-artist-ratings-v1_0.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

CPU times: user 17.9 s, sys: 3.63 s, total: 21.5 s
Wall time: 21.5 s


In [4]:
%%time

# Cleanup original dataset from Yahoo
# Removes ratings of 255 (which means never play again)[actual ratings range from 0 to 100]
# Removes items -100 and -99 (Unknown Artist and Not Applicable)
# Number of entries will be reduced from 115,579,440 to 115,248,575 and then to 115,248,573
# "Normalize" item ids (They go from ranging between -100 and 1,101,749 to ranging from 1 to 98,213)
fileIN = open("ydata-ymusic-user-artist-ratings-v1_0.txt", "r+")
fileOUT = open("ydatamusic.txt", "w+")

map = {}
n = 1
for line in fileIN:
        token = line.strip().split()
        uid = token[0]
        iid = token[1]
        score = token[2]
        if int(score) != 255 and int(iid) > 0:
            if iid in map:
                real_iid = map[iid]
            else:
                map[iid] = n
                real_iid = n
                n+=1
            fileOUT.write(str(uid))
            fileOUT.write("\t")
            fileOUT.write(str(real_iid))
            fileOUT.write("\t")
            fileOUT.write(str(score))
            fileOUT.write("\n")
fileIN.close()
fileOUT.close()

CPU times: user 23min 59s, sys: 6.88 s, total: 24min 6s
Wall time: 24min 31s


In [5]:
%%time

data_file = "ydatamusic.txt"

m = 1_948_882
n = 98_211
nnz_train = 103_723_717
nnz_test = 11_524_856

all_data = pd.read_csv(data_file, delimiter="\t", header=None)
user = all_data[0].values
item = all_data[1].values
rating = all_data[2].values

CPU times: user 51 s, sys: 5.92 s, total: 57 s
Wall time: 50.5 s


In [6]:
%%time

os.remove("ydata-ymusic-user-artist-ratings-v1_0.txt.gz")
os.remove("ydata-ymusic-user-artist-ratings-v1_0.txt")
os.remove("ydata-ymusic-artist-names-v1_0.txt.gz")
os.remove("ydatamusic.txt")

CPU times: user 795 ms, sys: 1.42 s, total: 2.21 s
Wall time: 1.42 s


In [7]:
%%time

print(user)
print(item)
print(rating)
print("")
print(np.min(user))
print(np.min(item))
print(np.min(rating))
print("")
print(np.max(user))
print(np.max(item))
print(np.max(rating))
print("")
print(np.unique(user).size)
print(np.unique(item).size)
print(np.unique(rating).size)
print("")
print(user.size)

assert np.max(user) == m
assert np.max(item) == n
assert user.size == nnz_train + nnz_test

[      1       1       1 ... 1948882 1948882 1948882]
[   1    2    3 ...  178  729 2191]
[ 90 100  90 ...  90  90  90]

1
1
0

1948882
98211
100

1947156
98211
101

115248573
CPU times: user 27.6 s, sys: 6.86 ms, total: 27.6 s
Wall time: 22.8 s


In [8]:
%%time

user_item = np.vstack((user, item))

user_item_train, user_item_test, rating_train, rating_test = train_test_split(user_item.T,
                                                                              rating,
                                                                              test_size=nnz_test,
                                                                              random_state=42)

CPU times: user 24 s, sys: 1.49 s, total: 25.5 s
Wall time: 23.5 s


In [9]:
%%time

# 1-based to 0-based
R_test_coo = sparse.coo_matrix((rating_test, (user_item_test[:, 0] - 1, user_item_test[:, 1] - 1)))
assert R_test_coo.nnz == nnz_test

outfile_test = open("test.txt", 'w')
for i in range(nnz_test):
    outfile_test.write(str((user_item_test[i, 0])) + " " + str((user_item_test[i, 1])) + " " + str(rating_test[i]) + "\n")

CPU times: user 1min 32s, sys: 441 ms, total: 1min 33s
Wall time: 1min 34s


In [10]:
%%time

# for test data, we need COO format to calculate test RMSE

R_test_coo.data.astype(np.float32).tofile('R_test_coo.data.bin')
R_test_coo.row.tofile('R_test_coo.row.bin')
R_test_coo.col.tofile('R_test_coo.col.bin')

test_data = np.fromfile('R_test_coo.data.bin', dtype=np.float32)
test_row = np.fromfile('R_test_coo.row.bin', dtype=np.int32)
test_col = np.fromfile('R_test_coo.col.bin', dtype=np.int32)

CPU times: user 24.1 ms, sys: 292 ms, total: 316 ms
Wall time: 348 ms


In [11]:
print(R_test_coo.data)
print(R_test_coo.row)
print(R_test_coo.col)
print("")
print(test_data)
print(test_row)
print(test_col)

[90 82 90 ...  0  0  0]
[1257724 1546778 1454022 ... 1371444 1713240 1691875]
[ 336   54  310 ... 5898  355  828]

[90. 82. 90. ...  0.  0.  0.]
[1257724 1546778 1454022 ... 1371444 1713240 1691875]
[ 336   54  310 ... 5898  355  828]


In [12]:
%%time

print(np.max(R_test_coo.data))
print(np.max(R_test_coo.row))
print(np.max(R_test_coo.col))
print("")
print(np.min(R_test_coo.data))
print(np.min(R_test_coo.row))
print(np.min(R_test_coo.col))
print("")
print(np.unique(user).size)
print(np.unique(R_test_coo.row).size)
print(np.unique(item).size)
print(np.unique(R_test_coo.col).size)

100
1948879
98205

0
0
0

1947156
1428803
98211
38802
CPU times: user 22.5 s, sys: 39.8 ms, total: 22.5 s
Wall time: 17.3 s


In [13]:
%%time

# 1-based to 0-based
R_train_coo = sparse.coo_matrix((rating_train, (user_item_train[:, 0] - 1, user_item_train[:, 1] - 1)))
assert R_train_coo.nnz == nnz_train

outfile_train = open("train.txt", 'w')
for i in range(nnz_train):
    outfile_train.write(str((user_item_train[i, 0])) + " " + str((user_item_train[i, 1])) + " " + str(rating_train[i]) + "\n")

CPU times: user 13min 46s, sys: 3.97 s, total: 13min 50s
Wall time: 13min 56s


In [14]:
%%time

# for training data, we need COO format to calculate training RMSE
# we need CSR format R when calculate X from \Theta
# we need CSC format of R when calculating \Theta from X
R_train_coo.data.astype(np.float32).tofile('R_train_coo.data.bin')
R_train_coo.row.tofile('R_train_coo.row.bin')
R_train_coo.col.tofile('R_train_coo.col.bin')

R_train_csr = R_train_coo.tocsr()
R_train_csc = R_train_coo.tocsc()

R_train_csr.data.astype(np.float32).tofile('R_train_csr.data.bin')
R_train_csr.indices.tofile('R_train_csr.indices.bin')
R_train_csr.indptr.tofile('R_train_csr.indptr.bin')
R_train_csc.data.astype(np.float32).tofile('R_train_csc.data.bin')
R_train_csc.indices.tofile('R_train_csc.indices.bin')
R_train_csc.indptr.tofile('R_train_csc.indptr.bin')

CPU times: user 29.4 s, sys: 5.63 s, total: 35 s
Wall time: 35.1 s


In [15]:
%%time

train_data = np.fromfile('R_train_coo.data.bin', dtype=np.float32)
train_row = np.fromfile('R_train_coo.row.bin', dtype=np.int32)
train_col = np.fromfile('R_train_coo.col.bin', dtype=np.int32)

train_csc_data = np.fromfile('R_train_csc.data.bin', dtype=np.float32)
train_csc_indices = np.fromfile('R_train_csc.indices.bin', dtype=np.int32)
train_csc_indptr = np.fromfile('R_train_csc.indptr.bin', dtype=np.int32)

train_csr_data = np.fromfile('R_train_csr.data.bin', dtype=np.float32)
train_csr_indices = np.fromfile('R_train_csr.indices.bin', dtype=np.int32)
train_csr_indptr = np.fromfile('R_train_csr.indptr.bin', dtype=np.int32)

CPU times: user 1.11 ms, sys: 2.81 s, total: 2.81 s
Wall time: 2.81 s


In [16]:
print(R_train_coo.data)
print(R_train_coo.row)
print(R_train_coo.col)
print("")
print(train_data)
print(train_row)
print(train_col)
print("")
print(R_train_csr.data)
print(R_train_csr.indices)
print(R_train_csr.indptr)
print("")
print(train_csr_data)
print(train_csr_indices)
print(train_csr_indptr)
print("")
print(R_train_csc.data)
print(R_train_csc.indices)
print(R_train_csc.indptr)
print("")
print(train_csc_data)
print(train_csc_indices)
print(train_csc_indptr)

[70  0 90 ... 60 80  0]
[ 645041  271612  494892 ...  958739  956552 1106630]
[3038  835  452 ...  214  339  410]

[70.  0. 90. ... 60. 80.  0.]
[ 645041  271612  494892 ...  958739  956552 1106630]
[3038  835  452 ...  214  339  410]

[ 90 100 100 ...  90  90  90]
[   0    1    3 ...  728  822 2190]
[        0        35        40 ... 103723709 103723711 103723717]

[ 90. 100. 100. ...  90.  90.  90.]
[   0    1    3 ...  728  822 2190]
[        0        35        40 ... 103723709 103723711 103723717]

[90 90 90 ...  0 30 50]
[      0       4       9 ... 1899090 1899090 1920258]
[        0    154716    228738 ... 103723715 103723716 103723717]

[90. 90. 90. ...  0. 30. 50.]
[      0       4       9 ... 1899090 1899090 1920258]
[        0    154716    228738 ... 103723715 103723716 103723717]


In [17]:
%%time

print(np.max(R_train_coo.data))
print(np.max(R_train_coo.row))
print(np.max(R_train_coo.col))
print("")
print(np.min(R_train_coo.data))
print(np.min(R_train_coo.row))
print(np.min(R_train_coo.col))
print("")
print(np.unique(user).size)
print(np.unique(R_train_coo.row).size)
print(np.unique(item).size)
print(np.unique(R_train_coo.col).size)

100
1948881
98210

0
0
0

1947156
1926443
98211
94086
CPU times: user 41.4 s, sys: 1.87 s, total: 43.2 s
Wall time: 37.2 s


In [18]:
%%time

print("writing extra meta_modified_all file")

outfile_meta = open("meta_modified_all", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n" + str(nnz_train) + "\n")
outfile_meta.write("""R_train_coo.data.bin
R_train_coo.row.bin
R_train_coo.col.bin
R_train_csr.indptr.bin
R_train_csr.indices.bin
R_train_csr.data.bin
R_train_csc.indptr.bin
R_train_csc.indices.bin
R_train_csc.data.bin
""")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

writing extra meta_modified_all file
CPU times: user 867 µs, sys: 0 ns, total: 867 µs
Wall time: 482 µs


In [19]:
%%time

print("writing extra meta file")

outfile_meta = open("meta", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n")
outfile_meta.write(str(nnz_train) + " " + "train.txt\n")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

writing extra meta file
CPU times: user 1.75 ms, sys: 1e+03 ns, total: 1.75 ms
Wall time: 1.26 ms
